In [ ]:

%load_ext autoreload
%autoreload 2

In [1]:
#Get modules
#indicators
from Incision import *
from Headcuts import *
from Erosion import *
from Invasives import *
from Bioassessment import *
from Conifer import *
from VegVigor import *
from AOP import *
from Habitat import *
from Ditches import *
from FormatStagingTables import *
from SEZScores import *



{'Angora Creek - tributary': 519, 'Angora Creek - upper': 520, 'Angora meadows - 1': 87, 'Angora meadows - 2': 90, 'Angora meadows - 3': 91, 'Angora meadows - 4': 143, 'Angora meadows - 5': 144, 'Angora meadows - 6': 142, 'Angora meadows - 7': 89, 'Angora meadows - 8': 88, 'Angora meadows - 9': 92, 'Angora meadows tributary - 1': 217, 'Angora meadows tributary - 2': 99, 'Angora meadows tributary - 3': 97, 'Angora meadows tributary - 4': 94, 'Angora meadows tributary - 5': 214, 'Angora meadows tributary - 6': 146, 'Angora meadows tributary - 7': 93, 'Angora meadows tributary - 8': 95, 'Angora meadows tributary - 9': 96, 'Angora tributary': 446, 'Antone meadows': 187, 'Baldwin marsh - 1': 160, 'Benwood meadows - 1': 129, 'Benwood meadows - 2': 131, 'Big Meadow - 1': 47, 'Big Meadow - 2': 48, 'Big Meadow - 3': 38, 'Big Meadow - 4': 39, 'Big Meadow - 5': 37, 'Big Meadow - 6': 40, 'Big meadow - 7': 126, 'Big Meadow Creek - lower': 521, 'Big Meadow Creek - upper': 491, 'Big Meadow Creek - up

# Process Indicators Post To Staging Tables

## Process when new Data is Available = External Data Sources

In [ ]:
#CONIFER#
#!!!No Need to update if there is no new information
#Current code grabs old data which is already in staging table
from Conifer import *
df=get_conifersez_data_sql()
readydf=process_conifer(df)
post_conifer_data(df, draft=True)

In [ ]:
#AOP#
#!!! No need to update if there is no new information- 
# code isn't complete for new data- do manually for now until we set up a script for sde.aquaticorganismpassage_usfs
#Current code grabs old data which is already in staging table
from AOP import *
#Get data from SEZ_Assessment Unit Table in sde (this indicator doesn't get updated much)
df = get_aopsez_data_sql()
#rename the sql data so that we can 
readydf = Process_aop(df)
#post_AOP_data(readydf, draft=True)

In [ ]:
#VEG VIGOR#
#Use data from SEZTable until we get new lidar data
df=get_sez_data_sql()
#df= get_oldvegvig_data()
post_Veg_data(df, draft=True)


In [ ]:
#HABITAT FRAGmentation and DITCHES
#LiDAR data? this shouldn't change much 
# we can use data in the sez_habfrag_score and sez_ditches_scores tables and manually change

## Process Every Year= In-house Monitoring Data

In [ ]:
#BIOASSESSMENT#done Update EVERY YEAR after Processing BMI Data and updating SDE.Stream
#Last updated 2023 
from Bioassessment import *
# from Bioassessment import *
df = get_bioassessment_data()
#clean, process, grade data
readydf = process_grade_bioassessment(df, 2023)
#post data to sde.VECTOR.stage headcut table or csv
post_bioassessment(readydf, draft=True)


In [ ]:
#INVASIVE SPECIES#done Update EVERY YEAR after QA of data in sde.Collect
#Last Updated 2023
from Invasives import *
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
Idf = get_combined_survey_and_invasive_data()
#clean, merge with external data/ format plant type names
df = merge_format_prioritize_invasive(Idf, usfsdf, 2024)
#Process Data and grade/rate assessment
#Be sure to update the priority list in "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority Lookup.csv"
#rate_invasives is in utils if you need to update that
invasive_summary, invasive_priority_summary = process_grade_invasives(df)
#Format Data so it is ready to be added to sde/csv for one last QA
readydf= final_format_invasive(df, invasive_priority_summary)
#post data to sde.VECTOR.stage headcut table or csv
post_invasive(readydf, draft=True)

In [ ]:
#HEADCUT#done#Update Every Year after QA of data in sde.Collect
#Last Updated 2023
from Headcuts import *
#Get raw headcut data prior 2024 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()
#Get new headcut data from sde collect
df = get_combined_survey_and_headcut_data()
#clean, process, grade data
readydf = process_grade_headcut(df, 2024)
#post data to sde.VECTOR.stage headcut table or csv
#draft = True is for QA and False is final and will be put into database
post_headcut(readydf, draft=False)


In [ ]:
#INCISION# Update Every Year after QA of data in sde.Collect
#Last Updated 2023
from Incision import *

#Get raw incision data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allincision_data()
#Get incision data from sde collect
df = get_combined_survey_and_incision_data()
#clean, process, grade data
readydf = process_grade_incision(df, 2022)
#post data to sde.VECTOR.stage incision table or csv
post_incision(readydf, draft=True)

In [ ]:
#EROSION# Update Every Year after QA of data in sde.Collect
#Last updated 2023
from Erosion import *

#Get raw erosion data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
df= get_erosion_data()

#clean, process, grade data
readydf = process_grade_erosion(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_erosion(readydf, draft=True)

# Staging Tables to Final Table

## 

In [2]:
from functools import reduce
from FormatStagingTables import *
# Staging to final
dfbanks = get_fs_data(bank_stability_url)
dfbiotic = get_fs_data(biotic_integrity_url)
dfconifer = get_fs_data(conifer_url)
dfditch = get_fs_data(ditches_url)
dfinvasive = get_fs_data(invasives_url)
dfhabitat = get_fs_data(Hab_Frag_url)
dfvegetation = get_fs_data(vegetation_url)
dfincision = get_fs_data(incision_url)
dfheadcuts = get_fs_data(headcuts_url)
dfAOP = get_fs_data(AOP_url)
dfSEZ = get_fs_data_spatial(SEZ_url)
dfheadcuts.drop(columns=['small', 'medium', 'large'], inplace=True)
#Format Staging Tables
#format biotic data average if two streams in one assessment unit
averaged_biotic_df=average_biotic_scores(dfbiotic)

#Same for meadow(large polygon) and riverine(small polygon) data drop these columns because not needed in final merge, will assign SEZ ID later
columns_to_drop = ['Year', 'SEZ_ID', 'GlobalID', 'last_edited_user', 'created_date', 'OBJECTID', 'created_user', 'last_edited_date']

#Create Dictionary of Dataframes to adjust year to be in datashource column and not its own column
yeartodatasource = {
    'dfbanks': dfbanks,
    'dfheadcuts': dfheadcuts,
    'dfincision': dfincision,
    'dfinvasive': dfinvasive
}

# Iterate over each DataFrame in meadowdata
for name, df in yeartodatasource.items():
    # Iterate over columns in the DataFrame
    for col in df.columns:
        # Check if the column name contains 'Data'
        if 'Data_' in col:
            # Add Year to the column if it contains 'Data'
            df[col] = df[col].astype(str) + ', ' + df['Year'].astype(str)

#Name dataframes so we can reference later
largepolygondata= {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Staging Tables Riverine/ small polygons
smallpolygondata = {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


# Process large polygon (meadow) and small polygon (riverine) data -
#Get most recent scores for each indicator
# Clean Data in each dataframe, assign SEZ ID to each assessment unit, add year to datasource column for formatting purposes
processed_largepolygon_data = process_data(largepolygondata, lookup_dict, columns_to_drop)
processed_smallpolygon_data = process_data(smallpolygondata, lookup_riverine, columns_to_drop)


# Merge large polygon and small polygon data
# Function to merge all DataFrames on multiple keys)
# # #moved to formatstagingtables.py
def merge_dataframes(data_dict, keys):
    return reduce(lambda left, right: pd.merge(left, right, on=keys, how='outer'), data_dict.values())


# # Merge small polygon DataFrames
smallpolygon_df = merge_dataframes(processed_smallpolygon_data, ['SEZ_ID', 'Assessment_Unit_Name' ])

# Merge large polygon DataFrames so all indicators that have data
largepolygon_df = merge_dataframes(processed_largepolygon_data, ['SEZ_ID', 'Assessment_Unit_Name' ])


# Append smallpolygon_df to largepolygon_df to have a complete list of all meadows and all sez ids
final_combined_df = pd.concat([largepolygon_df, smallpolygon_df], ignore_index=True)

# Print the final combined DataFrame to check
#print("Final Combined DataFrame:")
print(final_combined_df)


# Join SEZinfo to combined df to get a dataframe with more info about each Assessment Unit
#Prep SEZ Baseline Data for assessment unit...will need to rethink if acreage changes.. or just manually change in sde
keep_columns = ['SHAPE', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']
#dfSEZ is assessment unit information from SDE?
dfSEZinfo= dfSEZ.loc[:,keep_columns].copy()
dfSEZ['SEZ_ID']= dfSEZ['SEZ_ID'].astype(int)
# Join SEZinfo to the combined_df using SEZ_ID
final_df = pd.merge(final_combined_df, dfSEZinfo, on='SEZ_ID')

#Assign Threshold Calculations which is the Threshold Year--> is just the most recent data within the past 4 years
final_df['Threshold_Year'] = 'test'

print(final_df)

Processing DataFrame: dfbanks
Columns: Index(['OBJECTID', 'Assessment_Unit_Name', 'SEZ_ID',
       'Bank_Stability_Data_Source', 'Bank_Stability_Percent_Unstable',
       'Bank_Stability_Rating', 'Bank_Stability_Score', 'Year', 'GlobalID',
       'created_user', 'created_date', 'last_edited_user', 'last_edited_date'],
      dtype='object')
Processing DataFrame: dfaveraged_biotic
Columns: Index(['Assessment_Unit_Name', 'Year', 'Biotic_Integrity_CSCI',
       'Biotic_Integrity_Data_Source', 'Biotic_Integrity_Rating',
       'Biotic_Integrity_Score'],
      dtype='object')
Processing DataFrame: dfconifer
Columns: Index(['OBJECTID', 'Assessment_Unit_Name', 'SEZ_ID',
       'Conifer_Encroachment_Data_Sourc', 'Conifer_Percent_Encroached',
       'Conifer_Encroachment_Rating', 'Conifer_Encroachment_Score', 'Year',
       'ConiferEncroachment_Comments', 'GlobalID', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date'],
      dtype='object')
Processing DataFrame: dfditc

In [3]:
# Join SEZinfo to combined df to get a dataframe with more info about each Assessment Unit
#Prep SEZ Baseline Data for assessment unit...will need to rethink if acreage changes.. or just manually change in sde
keep_columns = ['SHAPE', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']
#dfSEZ is assessment unit information from SDE?
dfSEZinfo= dfSEZ.loc[:,keep_columns].copy()
dfSEZ['SEZ_ID']= dfSEZ['SEZ_ID'].astype(int)
# Join SEZinfo to the combined_df using SEZ_ID
final_df = pd.merge(final_combined_df, dfSEZinfo, on='SEZ_ID')

#Assign Threshold Calculations which is the Threshold Year--> is just the most recent data within the past 4 years
final_df['Threshold_Year'] = 'test'

#Bring in Last year threshold scores for any indicators thataren't in our raw data- some bank stability came from our stream surveys but aren't in the data
#Fill in any indicators that have missing data with data from 2019
# Field Mapping so 2019 threshol data renamed so it can be joined to new data
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'Threshold_Year': 'Threshold Year',
    'Comments': 'Comments',
    'SEZ_Type': 'SEZ_Type',
    'SEZ_ID': 'SEZ_ID',
    'Acres': 'Acres',
    'AquaticOrganismPassage_Barriers': 'AOP_BarriersPerMile',
    'AquaticOrganismPassage_NumberOf': 'AOP_NumberofBarriers',
    'AquaticOrganismPassage_Score': 'AOP_Score',
    'AquaticOrganismPassage_Rating': 'AOP_Rating',
    'AquaticOrganismPassage_StreamMi': 'AOP_StreamMiles',
    'AquaticOrganismPassage_DataSour': 'AOP_DataSource',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'Biotic_Integrity_Rating': 'Biotic_Integrity_Rating',
    'Biotic_Integrity_CSCI': 'Biotic_Integrity_CSCI',
    'Biotic_Integrity_Data_Source': 'Biotic_Integrity_Data_Source',
    'Biotic_Integrity_Score': 'Biotic_Integrity_Score',
    'Conifer_Encroachment_Percent_En': 'Conifer_Percent_Encroached',
    'Conifer_Encroachment_Data_Sourc': 'Conifer_Encroachment_Data_Sourc',
    'Conifer_Encroachment_Rating': 'Conifer_Encroachment_Rating',
    'Conifer_Encroachment_Score': 'Conifer_Encroachment_Score',
    'ConiferEncroachment_Comments': 'ConiferEncroachment_Comments',
    'Ditches_Data_Source': 'Ditches_Data_Source',
    'Ditches_Length': 'Ditches_Length',
    'Ditches_Meadow_Length': 'Ditches_Meadow_Length',
    'Ditches_Percent': 'Ditches_Percent',
    'Ditches_Rating': 'Ditches_Rating',
    'Ditches_Score': 'Ditches_Score',
    'Feature_Type': 'Feature_Type',
    'Habitat_Fragmentation_Data_Sour': 'Habitat_Frag_Data_Source',
    'Habitat_Fragmentation_Imperviou': 'Habitat_Frag_Impervious_Acres',
    'Habitat_Fragmentation_Percent_I': 'Habitat_Frag_Percent_Impervious',
    'Habitat_Fragmentation_Rating': 'Habitat_Frag_Rating',
    'Habitat_Fragmentation_Score': 'Habitat_Frag_Score',
    'Headcuts_Data_Source': 'Headcuts_Data_Source',
    'Headcuts_Number_of_Headcuts':'Number_of_Headcuts',
    'Headcuts_Rating': 'Headcuts_Rating',
    'Headcuts_Score': 'Headcuts_Score',
    'Incision_Data_Source': 'Incision_Data_Source',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'Incision_Ratio': 'Incision_Ratio',
    'Invasive_Percent_Cover': 'Invasive_Percent_Cover',
    'Invasive_Rating': 'Invasives_Rating',
    'Invasives_Data_Source': 'Invasives_Data_Source',
    'Invasives_Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Plant_Types': 'Invasives_Plant_Types',
    'Invasives_Scores': 'Invasives_Scores',
    'NDVI_ID': 'NDVI_ID',
    'Ownership_Primary': 'Ownership_Primary',
    'Ownership_Secondary': 'Ownership_Secondary',
    'Ownership_Secondary_2': 'Ownership_Secondary_2',
    'Ownership_Secondary_3': 'Ownership_Secondary_3',
    'VegetationVigor_DataSource': 'VegetationVigor_DataSource',
    'VegetationVigor_Rating': 'VegetationVigor_Rating',
    'VegetationVigor_Raw': 'VegetationVigor_Raw',
    'VegetationVigor_Score': 'VegetationVigor_Score'
}

# Rename fields based on field mappings
dfSEZprep = dfSEZ.rename(columns=field_mapping)
# Filter out columns in dfSEZprep that are not in final_df
dfSEZprep = dfSEZprep[[col for col in final_df.columns if col in dfSEZprep.columns]]
# # Ensure dfSEZprep has the same columns as final_df
#dfSEZprep = dfSEZprep[final_df.columns]

# Set SEZ_ID as index for both dataframes
final_df.set_index('SEZ_ID', inplace=True)
dfSEZprep.set_index('SEZ_ID', inplace=True)

# Update missing values in final_df using values from dfSEZprep
#final_df.update(dfSEZprep)
#final_df = final_df.fillna(dfSEZprep)
# Reset the index to get SEZ_ID back as a column
#final_df.reset_index(inplace=True)

 # Combine DataFrames to fill missing values
final_df = final_df.combine_first(dfSEZprep)

# Reset the index to get SEZ_ID back as a column
final_df.reset_index(inplace=True)

print(final_df)


      SEZ_ID  AOP_BarriersPerMile          AOP_DataSource  \
0          1                  0.0  USFS / TRPA 2009, 2019   
1          1                  0.0  USFS / TRPA 2009, 2019   
2          1                  0.0  USFS / TRPA 2009, 2019   
3          1                  0.0  USFS / TRPA 2009, 2019   
4          2                  0.0  USFS / TRPA 2009, 2019   
...      ...                  ...                     ...   
2559     642                 10.0  USFS / TRPA 2009, 2019   
2560     643                 10.0  USFS / TRPA 2009, 2019   
2561     643                 10.0  USFS / TRPA 2009, 2019   
2562     643                 10.0  USFS / TRPA 2009, 2019   
2563     643                 10.0  USFS / TRPA 2009, 2019   

      AOP_NumberofBarriers AOP_Rating  AOP_Score  AOP_StreamMiles      Acres  \
0                      0.0          A       12.0             <NA>  16.661797   
1                      0.0          A       12.0             <NA>  16.661797   
2                      0.0 

## QA tool

In [ ]:
#-------------
#QAOnly
#------------
# #RUN FOR QA so you can see the resulting dataframe/scores/formatting etc.
#CSV will be posteed to the SEZ_scripts folder in file explorer
# List of columns to export (excluding 'geometry' or 'Shape')
columns_to_export = [col for col in final_df.columns if col != 'SHAPE']
final_df[columns_to_export].to_csv('ready_df.csv', index=False)

# Calcluate SEZ Quality based on SEZ Type

In [4]:
from SEZScores import *
#Calculate Score based on SEZ_Type  #Future- make it so any indicator that isn't in the lists below for SEZ Type says NA
#Use SEZ_Type to select only needed indicators for SEZ Type
# Define the score columns needed for each SEZ Type
# Function to get the score columns based on SEZ_Type


# Apply the appropriate score columns based on SEZ_Type
final_df['Score_Columns'] = final_df['SEZ_Type'].apply(get_score_columns)


# Apply the score calculation to each row
final_df[['Final_Total_Points', 'Final_Points_Possible']] = final_df.apply(calculate_scores, axis=1)

# Calculate the final percent
final_df['Final_Percent'] = final_df['Final_Total_Points'] / final_df['Final_Points_Possible']

# Calculate the final rating and score
final_df['Final_Rating'] = final_df['Final_Percent'].apply(rate_SEZ)
final_df['Final_Score'] = final_df['Final_Rating'].apply(score_indicator)

# Drop the temporary 'Score_Columns' column
final_df = final_df.drop(columns=['Score_Columns'])
print(final_df)



      SEZ_ID  AOP_BarriersPerMile          AOP_DataSource  \
0          1                  0.0  USFS / TRPA 2009, 2019   
1          1                  0.0  USFS / TRPA 2009, 2019   
2          1                  0.0  USFS / TRPA 2009, 2019   
3          1                  0.0  USFS / TRPA 2009, 2019   
4          2                  0.0  USFS / TRPA 2009, 2019   
...      ...                  ...                     ...   
2559     642                 10.0  USFS / TRPA 2009, 2019   
2560     643                 10.0  USFS / TRPA 2009, 2019   
2561     643                 10.0  USFS / TRPA 2009, 2019   
2562     643                 10.0  USFS / TRPA 2009, 2019   
2563     643                 10.0  USFS / TRPA 2009, 2019   

      AOP_NumberofBarriers AOP_Rating  AOP_Score  AOP_StreamMiles      Acres  \
0                      0.0          A       12.0             <NA>  16.661797   
1                      0.0          A       12.0             <NA>  16.661797   
2                      0.0 